# 🚀 Bitcoin Real-Time Analytics Live Demo

## Comprehensive showcase of the entire data pipeline

**⚠️ Known Issue & Fix:**
- **Problem**: The API shows `model_loaded: false` because Spark hasn't trained the model yet
- **Root Cause**: Producer sends all data at once then exits, Spark needs continuous streaming
- **Solution**: Run the cells below to verify the pipeline and manually trigger model training

This notebook demonstrates:
- ✅ Docker services status
- ✅ Live Kafka messages  
- ✅ ML model training metrics
- ✅ API predictions
- ✅ Real-time data visualizations
- ✅ Troubleshooting common issues

## 1️⃣ Check Docker Services Status

In [2]:
import subprocess
import json
import pandas as pd
from datetime import datetime
import requests
import time

def run_command(cmd):
    """Execute shell command and return output"""
    result = subprocess.run(cmd, shell=True, capture_output=True, text=True)
    return result.stdout + result.stderr

# Check Docker services
print("📊 Docker Services Status")
print("=" * 50)
output = run_command("docker compose ps")
print(output)

📊 Docker Services Status
NAME                 IMAGE                                        COMMAND                  SERVICE           CREATED         STATUS                   PORTS
bitcoin-api          bitcoin-realtime-analytics-api               "uvicorn api_service…"   api               7 minutes ago   Up 6 minutes             0.0.0.0:8000->8000/tcp, [::]:8000->8000/tcp
kafka                confluentinc/cp-kafka:7.5.0                  "/etc/confluent/dock…"   kafka             7 minutes ago   Up 7 minutes (healthy)   0.0.0.0:9092-9093->9092-9093/tcp, [::]:9092-9093->9092-9093/tcp
kafka-ui             provectuslabs/kafka-ui:latest                "/bin/sh -c 'java --…"   kafka-ui          7 minutes ago   Up 7 minutes             0.0.0.0:8080->8080/tcp, [::]:8080->8080/tcp
realtime-dashboard   bitcoin-realtime-analytics-dashboard         "streamlit run realt…"   dashboard         7 minutes ago   Up 6 minutes             0.0.0.0:8501->8501/tcp, [::]:8501->8501/tcp
spark-streaming      bi

## 2️⃣ Monitor Kafka Producer (Data Ingestion)

In [3]:
print("📨 Bitcoin Producer Activity (Last 20 lines)")
print("=" * 50)
logs = run_command("docker compose logs producer | tail -20")
print(logs)

📨 Bitcoin Producer Activity (Last 20 lines)
bitcoin-producer  | ➡ Sent: {'date': '2026-01-04 05:00:00', 'open': 91349.1484375, 'high': 91438.0, 'low': 91317.9609375, 'close': 91432.5703125, 'volume': 0}
bitcoin-producer  | ➡ Sent: {'date': '2026-01-04 06:00:00', 'open': 91434.1328125, 'high': 91685.1796875, 'low': 91213.8046875, 'close': 91213.8046875, 'volume': 101595136}
bitcoin-producer  | ➡ Sent: {'date': '2026-01-04 07:00:00', 'open': 91206.6875, 'high': 91474.9375, 'low': 91206.6875, 'close': 91474.9375, 'volume': 42860544}
bitcoin-producer  | ➡ Sent: {'date': '2026-01-04 08:00:00', 'open': 91487.390625, 'high': 91487.390625, 'low': 91176.1171875, 'close': 91307.09375, 'volume': 265007104}
bitcoin-producer  | ➡ Sent: {'date': '2026-01-04 09:00:00', 'open': 91327.6875, 'high': 91426.203125, 'low': 91244.1015625, 'close': 91409.796875, 'volume': 814034944}
bitcoin-producer  | ➡ Sent: {'date': '2026-01-04 10:00:00', 'open': 91418.1171875, 'high': 91554.625, 'low': 91387.0234375, 'cl

## 3️⃣ Monitor Spark Streaming (ML Processing)

In [4]:
print("🎯 Spark Streaming Activity (Last 20 lines)")
print("=" * 50)
logs = run_command("docker compose logs spark-streaming | tail -20")
print(logs)

🎯 Spark Streaming Activity (Last 20 lines)
spark-streaming  | 	---------------------------------------------------------------------
spark-streaming  | 	|                  |            modules            ||   artifacts   |
spark-streaming  | 	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
spark-streaming  | 	---------------------------------------------------------------------
spark-streaming  | 	|      default     |   11  |   11  |   11  |   0   ||   11  |   11  |
spark-streaming  | 	---------------------------------------------------------------------
spark-streaming  | :: retrieving :: org.apache.spark#spark-submit-parent-1e60ec4a-9f3d-4d3d-93ee-614f20fbd7f0
spark-streaming  | 	confs: [default]
spark-streaming  | 	11 artifacts copied, 0 already retrieved (62936kB/349ms)
spark-streaming  | 26/01/04 22:11:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
spark-streaming  | Using Spark'

## 4️⃣ Check ML Model Metrics

In [5]:
import os

print("🤖 ML Model Training Metrics")
print("=" * 50)

metrics_file = "/home/azureuser/bitcoin_spark/bitcoin-realtime-analytics/model_metrics.txt"

if os.path.exists(metrics_file):
    with open(metrics_file, 'r') as f:
        metrics = f.read()
    print(metrics)
    print("\n✅ Model metrics are being generated in real-time!")
else:
    print("⏳ Metrics file still being generated by Spark Streaming...")

🤖 ML Model Training Metrics
Model: GradientBoosting Regressor
RMSE: 309.577502769582
MAE: 201.99444570937837
Trees: 20
Max Depth: 5
Feature Importances: [0.01866401420771625, 0.08788473715447968, 0.8827465191215687, 0.010704729516235311]
Batch: 4
Timestamp: 2026-01-04T22:13:33.326503


✅ Model metrics are being generated in real-time!


## 5️⃣ Check Metrics History (JSON Format)

In [6]:
import json
import pandas as pd

metrics_history_file = "/home/azureuser/bitcoin_spark/bitcoin-realtime-analytics/metrics_history.json"

if os.path.exists(metrics_history_file):
    with open(metrics_history_file, 'r') as f:
        try:
            history = json.load(f)
            
            # Display as DataFrame
            if isinstance(history, list) and len(history) > 0:
                df_metrics = pd.DataFrame(history)
                print("📈 Model Performance History (Last 10 updates)")
                print("=" * 80)
                print(df_metrics.tail(10).to_string())
                print(f"\n✅ Total training iterations: {len(history)}")
            else:
                print("⏳ Metrics history is accumulating...")
        except json.JSONDecodeError:
            print("⏳ Metrics file is still being written...")
else:
    print("⏳ Metrics history file not yet created...")

📈 Model Performance History (Last 10 updates)
    batch        rmse         mae  trees                                                                        feature_importances                   timestamp
37      0  986.639303  640.808466     20   [0.0006534188413004734, 0.08807232731935773, 0.9111622379338252, 0.00011201590551653004]  2026-01-04T14:51:23.564385
38      1  696.441206  453.691230     20  [0.00028060492918761335, 0.950709227352462, 0.048824481457239624, 0.00018568626111092497]  2026-01-04T14:51:41.282262
39      2  328.482755  211.402530     20       [0.005229023024365457, 0.7481584470175465, 0.23782596996101538, 0.00878655999707271]  2026-01-04T14:51:52.769603
40      3  564.979630  381.917351     20    [0.0012024750061918597, 0.05168384275959068, 0.9446254348134879, 0.0024882474207294386]  2026-01-04T14:52:03.005870
41      4  251.462887  171.003612     20      [0.014265010662428208, 0.07123713337155095, 0.9009093946233449, 0.013588461342676019]  2026-01-04T14:52:14.5

## 6️⃣ Test the REST API

In [7]:
import requests
import json
from datetime import datetime

API_URL = "http://localhost:8000"

print("🌐 Testing REST API")
print("=" * 50)

try:
    # Test health endpoint
    response = requests.get(f"{API_URL}/", timeout=5)
    print(f"✅ API Status: {response.status_code}")
    print(f"Response: {response.json()}")
    print()
    
    # Test predict endpoint
    response = requests.get(f"{API_URL}/predict", timeout=5)
    if response.status_code == 200:
        prediction = response.json()
        print("🎯 Latest Prediction from API:")
        print(json.dumps(prediction, indent=2))
    else:
        print(f"⚠️ Prediction endpoint returned: {response.status_code}")
        
except requests.exceptions.ConnectionError:
    print("⏳ API not yet available. Please wait for services to start...")
except Exception as e:
    print(f"⚠️ API Error: {str(e)}")

🌐 Testing REST API
✅ API Status: 200
Response: {'message': 'Bitcoin Price Prediction API', 'status': 'running', 'model_loaded': False, 'endpoints': {'predict': '/predict', 'batch_predict': '/batch_predict', 'model_info': '/model/info', 'reload_model': '/model/reload', 'health': '/health'}}

⚠️ Prediction endpoint returned: 405


## 7️⃣ View Real-Time Data Pipeline Statistics

In [8]:
print("📊 Real-Time Pipeline Statistics")
print("=" * 50)

# Get producer statistics
producer_logs = run_command("docker compose logs producer | grep -E '(Total records|Successfully)' | head -5")
print("📨 Producer:")
print(producer_logs if producer_logs else "⏳ No data yet...")
print()

# Get Spark statistics
spark_logs = run_command("docker compose logs spark-streaming | grep -E '(records|training|RMSE|MAE)' | tail -5")
print("🎯 Spark ML:")
print(spark_logs if spark_logs else "⏳ No data yet...")

📊 Real-Time Pipeline Statistics
📨 Producer:
bitcoin-producer  | ✅ Successfully connected to Kafka at kafka:29092
bitcoin-producer  |  Total records to send: 17511


🎯 Spark ML:
⏳ No data yet...


## 8️⃣ Quick Access Links

In [9]:
print("\n🌐 OPEN THESE IN YOUR BROWSER:")
print("=" * 50)
print()
print("1. 📊 Kafka UI (View live messages):")
print("   http://localhost:8080")
print()
print("2. 📈 Real-Time Dashboard:")
print("   http://localhost:8501")
print()
print("3. 🔧 API Documentation (Interactive):")
print("   http://localhost:8000/docs")
print()
print("4. 📋 OpenAPI Schema:")
print("   http://localhost:8000/openapi.json")
print()
print("=" * 50)


🌐 OPEN THESE IN YOUR BROWSER:

1. 📊 Kafka UI (View live messages):
   http://localhost:8080

2. 📈 Real-Time Dashboard:
   http://localhost:8501

3. 🔧 API Documentation (Interactive):
   http://localhost:8000/docs

4. 📋 OpenAPI Schema:
   http://localhost:8000/openapi.json



## 9️⃣ System Architecture Overview

In [10]:
print("""
╔════════════════════════════════════════════════════════════════════╗
║           BITCOIN REAL-TIME ANALYTICS ARCHITECTURE                 ║
╚════════════════════════════════════════════════════════════════════╝

┌─────────────────────────────────────────────────────────────────────┐
│ 1. DATA INGESTION (bitcoin_producer.py)                             │
│    ├─ Downloads BTC-USD data from yfinance                          │
│    ├─ Publishes to Kafka topic: bitcoin_data                        │
│    └─ Runs every interval (default: hourly)                         │
└─────────────────────────────────────────────────────────────────────┘
           │
           ▼
┌─────────────────────────────────────────────────────────────────────┐
│ 2. MESSAGE BROKER (Apache Kafka)                                    │
│    ├─ Zookeeper: localhost:2181                                     │
│    ├─ Kafka: localhost:9092 (external) / kafka:29092 (internal)     │
│    └─ Kafka UI: http://localhost:8080                               │
└─────────────────────────────────────────────────────────────────────┘
           │
           ▼
┌─────────────────────────────────────────────────────────────────────┐
│ 3. STREAM PROCESSING (spark_realtime_ml.py)                         │
│    ├─ Reads from Kafka topics                                       │
│    ├─ Random Forest ML model training                               │
│    ├─ Real-time prediction generation                               │
│    └─ Metrics: RMSE, MAE, R²                                        │
└─────────────────────────────────────────────────────────────────────┘
           │
           ▼
┌─────────────────────────────────────────────────────────────────────┐
│ 4. OUTPUT LAYER                                                     │
│    ├─ API (FastAPI): http://localhost:8000/docs                    │
│    ├─ Dashboard (Streamlit): http://localhost:8501                 │
│    └─ Metrics Files: model_metrics.txt, metrics_history.json        │
└─────────────────────────────────────────────────────────────────────┘

✅ All components are containerized and orchestrated with Docker Compose
""")


╔════════════════════════════════════════════════════════════════════╗
║           BITCOIN REAL-TIME ANALYTICS ARCHITECTURE                 ║
╚════════════════════════════════════════════════════════════════════╝

┌─────────────────────────────────────────────────────────────────────┐
│ 1. DATA INGESTION (bitcoin_producer.py)                             │
│    ├─ Downloads BTC-USD data from yfinance                          │
│    ├─ Publishes to Kafka topic: bitcoin_data                        │
│    └─ Runs every interval (default: hourly)                         │
└─────────────────────────────────────────────────────────────────────┘
           │
           ▼
┌─────────────────────────────────────────────────────────────────────┐
│ 2. MESSAGE BROKER (Apache Kafka)                                    │
│    ├─ Zookeeper: localhost:2181                                     │
│    ├─ Kafka: localhost:9092 (external) / kafka:29092 (internal)     │
│    └─ Kafka UI: http://localhost:8080

## 🔟 Troubleshooting & Status Check

## 🔟 Model Loading Troubleshooting

In [12]:
print("🔧 Full System Health Check")
print("=" * 50)

# Docker status
print("\n1. Docker Services:")
status = run_command("docker compose ps --format 'table {{.Names}}\\t{{.Status}}'")
print(status)

# Network check
print("\n2. Network Connectivity:")
kafka_check = run_command("docker compose exec -T kafka kafka-broker-api-versions --bootstrap-server localhost:29092 2>&1 | head -3")
print(f"Kafka connectivity: {'✅' if 'ApiVersion' in kafka_check else '⏳ Starting...'}")

# File system check
print("\n3. Generated Files:")
files_to_check = [
    "model_metrics.txt",
    "metrics_history.json",
    "checkpoint/"
]

base_path = "/home/azureuser/bitcoin_spark/bitcoin-realtime-analytics"
for file in files_to_check:
    full_path = os.path.join(base_path, file)
    if os.path.exists(full_path):
        if os.path.isdir(full_path):
            count = len(os.listdir(full_path))
            print(f"   ✅ {file}: {count} items")
        else:
            size = os.path.getsize(full_path) / 1024
            print(f"   ✅ {file}: {size:.2f} KB")
    else:
        print(f"   ⏳ {file}: Pending...")

print("\n" + "=" * 50)
print("💡 If services are not ready, wait 30-60 seconds and re-run this cell.")

🔧 Full System Health Check

1. Docker Services:
<no value>           STATUS
bitcoin-api          Up 17 seconds
kafka                Up 11 minutes (healthy)
kafka-ui             Up 11 minutes
realtime-dashboard   Up 11 minutes
spark-streaming      Up 11 minutes
zookeeper            Up 11 minutes


2. Network Connectivity:
Kafka connectivity: ⏳ Starting...

3. Generated Files:
   ✅ model_metrics.txt: 0.25 KB
   ✅ metrics_history.json: 13.54 KB
   ✅ checkpoint/: 91 items

💡 If services are not ready, wait 30-60 seconds and re-run this cell.


In [ ]:
print("🔍 Model Loading Troubleshooting")
print("=" * 50)

base_path = "/home/azureuser/bitcoin_spark/bitcoin-realtime-analytics"

# Check if model exists
model_path = os.path.join(base_path, "bitcoin_model")
if os.path.exists(model_path):
    print(f"\n✅ Model directory exists: {model_path}")
    
    # Check model contents
    model_contents = os.listdir(model_path)
    print(f"   Contents: {', '.join(model_contents)}")
    
    # Check model type
    metadata_path = os.path.join(model_path, "metadata")
    if os.path.exists(metadata_path):
        print(f"\n📋 Model Type: Gradient Boosting Tree (GBT)")
        print(f"   Structure:")
        for root, dirs, files in os.walk(model_path):
            level = root.replace(model_path, '').count(os.sep)
            indent = ' ' * 2 * level
            print(f'{indent}{os.path.basename(root)}/')
            subindent = ' ' * 2 * (level + 1)
            for file in files[:3]:  # Show first 3 files per dir
                print(f'{subindent}{file}')
            if len(files) > 3:
                print(f'{subindent}... and {len(files) - 3} more files')
else:
    print(f"\n❌ Model directory NOT found: {model_path}")
    print("\n💡 To fix:")
    print("   1. Ensure Spark streaming is running and has received data")
    print("   2. Check: docker compose logs spark-streaming")
    print("   3. Producer must send data to 'bitcoin_prices' topic")

# Check Spark streaming status
print("\n\n📊 Spark Streaming Status:")
spark_logs = run_command("docker compose logs spark-streaming 2>&1 | grep -E '(📊|✅|Batch|Training)' | tail -10")
if spark_logs.strip():
    print(spark_logs)
else:
    print("   ⚠️ No training activity detected")
    print("   💡 Spark may still be initializing or waiting for data")

# Test API model reload
print("\n\n🔄 Attempting to reload model via API...")
try:
    response = requests.post("http://localhost:8000/model/reload", timeout=5)
    result = response.json()
    print(f"   Result: {json.dumps(result, indent=2)}")
    
    if not result.get("success"):
        print("\n❌ Model reload failed")
        print("\n🔧 Quick Fix:")
        print("   1. Wait 2-3 minutes for Spark to finish training")
        print("   2. Re-run this cell")
        print("   3. If still failing, restart: docker compose restart spark-streaming")
except Exception as e:
    print(f"   ⚠️ API Error: {str(e)}")

## ✨ Summary

This notebook provides a comprehensive live demonstration of the Bitcoin Real-Time Analytics project:

✅ **Data Pipeline**: Real Bitcoin data → Kafka → Spark ML → Predictions  
✅ **Services**: All containerized and auto-starting  
✅ **Monitoring**: Live logs and metrics tracking  
✅ **APIs**: RESTful API with OpenAPI docs  
✅ **Visualization**: Interactive dashboard and Kafka UI  

### Next Steps:
1. **Re-run all cells** to refresh the status
2. **Open the browser links** to see live data flowing
3. **Check metrics** improving as more data arrives
4. **Test API** with custom parameters via the swagger UI